# 文本改写，从使用提示语开始


In [ ]:
# pip install openai
import openai;
def make_text_short(text):
    messages = []
    messages.append( {"role": "system", "content": "你是一个用来将文本改写得短的AI助手，用户输入一段文本，你给出一段意思相同，但是短小精悍的结果"})
    messages.append( {"role": "user", "content": text})
    response = openai.ChatCompletion.create(
        model="gpt-3.5-turbo",
        messages=messages,
        temperature=0.5,
        max_tokens=2048,
        presence_penalty=0,
        frequency_penalty=2,
        n=3,
    )
    return response

long_text = """
在这个快节奏的现代社会中，我们每个人都面临着各种各样的挑战和困难。
在这些挑战和困难中，有些是由外部因素引起的，例如经济萧条、全球变暖和自然灾害等。
还有一些是由内部因素引起的，例如情感问题、健康问题和自我怀疑等。
面对这些挑战和困难，我们需要采取积极的态度和行动来克服它们。
这意味着我们必须具备坚韧不拔的意志和创造性思维，以及寻求外部支持的能力。
只有这样，我们才能真正地实现自己的潜力并取得成功。
"""
short_version = make_text_short(long_text)

index = 1
for choice in short_version["choices"]:
    print(f"version {index}: " + choice["message"]["content"])
    index += 1

version 1: 现代社会中，我们面临各种挑战和困难。有些由外部因素引起，如经济萧条、全球变暖和自然灾害；还有些由内部因素引起，如情感问题、健康问题和自我怀疑等。要克服这些挑战，需要积极应对并具备坚韧不拔的意志、创造性思维以及寻求外部支持的能力。只有这样才能实现潜力并取得成功。
version 2: 现代社会充满挑战，有些来自外部如经济、环境问题，有些源于内在如情感、健康和自我怀疑。我们需要积极应对这些困难，展现坚韧的意志和创造性思维，并寻求外部支持以实现成功。
version 3: 在快节奏的现代社会中，我们面临各种挑战和困难。有些是由外部因素引起的（如经济萧条、全球变暖和自然灾害），而其他一些则源于内部问题（如情感、健康和自我怀疑）。要克服这些问题，需要积极应对并具备坚韧不拔的意志与创造性思维，并寻求外界支持。只有这样才能实现潜力并获得成功。


# 通过 logit_bias 参数精确控制内容

In [3]:
!pip install tiktoken
import tiktoken;
encoding = tiktoken.get_encoding('p50k_base')
token_ids = encoding.encode("灾害")
print(token_ids)

bias_map = {}
for token_id in token_ids:
    bias_map[token_id] = -100

def make_text_short(text):
    messages = []
    messages.append( {"role": "system", "content": "你是一个用来将文本改写得短的AI助手，用户输入一段文本，你给出一段意思相同，但是短小精悍的结果"})
    messages.append( {"role": "user", "content": text})
    response = openai.ChatCompletion.create(
        model="gpt-3.5-turbo", messages=messages, temperature=0.5, max_tokens=2048,
        n=3, presence_penalty=0, frequency_penalty=2, 
        logit_bias = bias_map,
    )
    return response

short_version = make_text_short(long_text)

index = 1
for choice in short_version["choices"]:
    print(f"version {index}: " + choice["message"]["content"])
    index += 1

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 19.8 MB/s eta 0:00:00
[163, 223, 122, 22522, 111]
version 1: 现代社会中，我们面对各种挑战和困难示弱。有内在因素如情感、健康问题和自我怀 疑，也有外部因素如经济萧条、全球变暖和自然災宝。要克服这些问题需要积枪态度与行动力，并具备坚韧不拔的意志与创造性思维能力以及寻求外部支持的能力。只有这样才能实现潜力并取得成功
version 2: 现代社会中，我们面临各种挑战和困难示例：经济萧条、全球变暖、情感问题等。要克服这些，需要积枝意志、创造性思维和外部支持。只有这样才能实现潜力并取得成功。
version 3: 现代社会中，人们面临各种挑战和困难示范出坚韧不拔的意志、创造性思维，并需要寻求外部支持来克服这些问题。只有这样，才能实现自己的潜力并取得成功。


# 使用英文来减少 Token 的使用

In [4]:
def translate(text):
    messages = []
    messages.append( {"role": "system", "content": "你是一个翻译，把用户的话翻译成英文"})
    messages.append( {"role": "user", "content": text})
    response = openai.ChatCompletion.create(
        model="gpt-3.5-turbo", messages=messages, temperature=0.5, max_tokens=2048,        n=1
    )
    return response["choices"][0]["message"]["content"]

chinese = long_text
english = translate(chinese)

num_of_tokens_in_chinese = len(encoding.encode(chinese))
num_of_tokens_in_english = len(encoding.encode(english))
print(english)
print(f"chinese: {num_of_tokens_in_chinese} tokens")
print(f"english: {num_of_tokens_in_english} tokens")

In this fast-paced modern society, each of us faces various challenges and difficulties. Some of these challenges and difficulties are caused by external factors such as economic downturn, global warming, and natural disasters. There are also some caused by internal factors such as emotional issues, health problems, and self-doubt. To overcome these challenges and difficulties, we need to adopt a positive attitude and take action. This means we must have a strong will and creative thinking, as well as the ability to seek external support. Only then can we truly realize our potential and achieve success.
chinese: 432 tokens
english: 114 tokens


# 看看 OpenAI 给了我们哪些模型

In [5]:
!pip install pandas;
import pandas as pd
# list all open ai models
engines = openai.Engine.list()
pd = pd.DataFrame(openai.Engine.list()['data'])
display(pd[['id', 'owner']])

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


,id,owner
0,babbage,openai
1,davinci,openai
2,text-davinci-edit-001,openai
3,gpt-3.5-turbo-0301,openai
4,babbage-code-search-code,openai-dev
5,text-similarity-babbage-001,openai-dev
6,gpt-3.5-turbo,openai
7,code-davinci-edit-001,openai
8,text-davinci-001,openai
9,ada,openai


In [6]:
from openai.embeddings_utils import get_embedding

text = "让我们来算算Embedding"

embedding_ada = get_embedding(text, engine="text-embedding-ada-002")
print("embedding-ada: ", len(embedding_ada))

similarity_ada = get_embedding(text, engine="text-similarity-ada-001")
print("similarity-ada: ", len(similarity_ada))

babbage_similarity = get_embedding(text, engine="babbage-similarity")
print("babbage-similarity: ", len(babbage_similarity))

babbage_search_query = get_embedding(text, engine="text-search-babbage-query-001")
print("search-babbage-query: ", len(babbage_search_query))

curie = get_embedding(text, engine="curie-similarity")
print("curie-similarity: ", len(curie))

davinci = get_embedding(text, engine="text-similarity-davinci-001")
print("davinci-similarity: ", len(davinci))

embedding-ada:  1536
similarity-ada:  1024
babbage-similarity:  2048
search-babbage-query:  2048
curie-similarity:  4096
davinci-similarity:  12288


# 插入内容，GPT 也可以像 BERT
use  openai.Completion.create with prefix and suffix params to insert text


In [7]:
prefix = """在这个快节奏的现代社会中，我们每个人都面临着各种各样的挑战和困难。
在这些挑战和困难中，有些是由外部因素引起的，例如经济萧条、全球变暖和自然灾害等。\n"""
# 还有一些是由内部因素引起的，例如情感问题、健康问题和自我怀疑等。
suffix = """\n面对这些挑战和困难，我们需要采取积极的态度和行动来克服它们。
这意味着我们必须具备坚韧不拔的意志和创造性思维，以及寻求外部支持的能力。
只有这样，我们才能真正地实现自己的潜力并取得成功。"""

def insert_text(prefix, suffix):
    response = openai.Completion.create(
        model="text-davinci-003",
        prompt=prefix,
        suffix=suffix,
        max_tokens=1024,
        )
    return response

response = insert_text(prefix, suffix)
print(response["choices"][0]["text"])

然而，也有一些是来自我们内心的挑战和困难，如恐惧、焦虑和自卑等情绪和弱点。



In [12]:

prefix = """在这个快节奏的现代社会中，我们每个人都面临着各种各样的挑战和困难。
在这些挑战和困难中，有些是由外部因素引起的，例如经济萧条、全球变暖和自然灾害等。\n"""
# 还有一些是由内部因素引起的，例如情感问题、健康问题和自我怀疑等。
suffix = """面对这些挑战和困难，我们需要采取积极的态度和行动来克服它们。这意味着我们必须具备坚韧不拔的意志和创造性思维，以及寻求外部支持的能力。只有这样，我们才能真正地实现自己的潜力并取得成功。"""

response = insert_text(prefix, suffix)
print(response["choices"][0]["text"])

而另一部分的挑战和困难是由个人因素，例如个人负担、压力和焦虑等。

无论是外部挑战还是个人因素，


# 不要乱问乱说，做个“正直”的 AI

In [13]:
def chatgpt(text):
    messages = []
    messages.append( {"role": "system", "content": "You are a useful AI assistant"})
    messages.append( {"role": "user", "content": text})
    response = openai.ChatCompletion.create(
        model="gpt-3.5-turbo",
        messages=messages,
        temperature=0.5,
        max_tokens=2048,
        top_p=1,
    )
    message = response["choices"][0]["message"]["content"]
    return message

threaten = "你不听我的我就拿刀砍死你"
print(chatgpt(threaten))

我是一个虚拟AI助手，没有身体，也不会感受到疼痛。我的唯一目的是帮助您解决问题和提供信息。如果您有任何需要帮助的问题，请随时问我。但请不要使用暴力或威胁语言，这不仅不合适，也不会得到任何帮助。


In [14]:
# use openai.Moderation.create to measure language sensitivity
threaten = "你不听我的我就拿刀砍死你"

def moderation(text):
    response = openai.Moderation.create(
        input=text
    )
    output = response["results"][0]
    return output
print(moderation(threaten))

{
  "categories": {
    "hate": false,
    "hate/threatening": false,
    "self-harm": false,
    "sexual": false,
    "sexual/minors": false,
    "violence": true,
    "violence/graphic": false
  },
  "category_scores": {
    "hate": 0.030033664777874947,
    "hate/threatening": 0.0002820899826474488,
    "self-harm": 0.004850226454436779,
    "sexual": 2.2907377569936216e-05,
    "sexual/minors": 6.477687275463495e-09,
    "violence": 0.9996402263641357,
    "violence/graphic": 4.35576839663554e-05
  },
  "flagged": true
}
